In [1]:
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
import datetime

%matplotlib inline

In [2]:
def con_2_twitter():
    CONSUMER_KEY = 'sNHWONz3Kxmef67nh2vj9lPk9'
    CONSUMER_SECRET = 'oRd3YfPMChU61158C3IzHuBl0inEtvxNrtLmsGtXY913Jk4LIh'
    ACCESS_TOKEN = '449910487-ren1xbe6RNBlmtd7K8EG6UJNvfRALCFltjXNi6Pa'
    ACCESS_SECRET = 'dOaWVZ3sq1d9FPUzOjyeNTWK2HByjIcCcAENDNgvfadvF'
    
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(auth)
    return api

def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [13]:
api = con_2_twitter()
tweets = api.get_status('1241678619266727941')
tweets._json

{'created_at': 'Sun Mar 22 10:50:26 +0000 2020',
 'id': 1241678619266727941,
 'id_str': '1241678619266727941',
 'text': 'Leuten auf Instagram folgen, weil man findet, dass sie schön aussehen/attraktiv sind. Was denkt ihr dazu?',
 'truncated': False,
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 482913109,
  'id_str': '482913109',
  'name': '✞addl',
  'screen_name': 'Taddl',
  'location': '',
  'description': 'TJ_beastboy & TJ_babybrain trapped in one body. 気 Management: kim@spacemgmt.de',
  'url': None,
  'entities': {'description': {'urls': []}},
  'protected': False,
  'followers_count': 1564529,
  'friends_count': 176,
  'listed_count': 547,
  'created_at': 'Sat Feb 04 12:53:2

In [6]:
api = con_2_twitter()
tweets = tweepy.Cursor(api.search,q='#Covid19',lang='de').items(1000)

df = pd.DataFrame(columns = ['text','user','len','ID','Date','Source','Like','RTs', 'Geo'])

for tweet in tweets:
    df = df.append({'text':tweet.text,'user':tweet.user.name,'len':len(tweet.text),'ID':tweet.id,'Date':tweet.created_at,'Source':tweet.source,'Like':tweet.favorite_count,'RTs':tweet.retweet_count, 'Geo': tweet.geo},ignore_index=True)

df.head()

,text,user,len,ID,Date,Source,Like,RTs,Geo
0,RT @MoritzWaech: Junge liberale: Öffentlich-Re...,Knusperpartner (im Homo Office 💻🏳️‍🌈),140,1241516705769283585,2020-03-22 00:07:03,Twitter for Mac,0,453,None
1,RT @Mau_Ri_83: #ItalienNews. Alles weist darau...,GigiV46,140,1241516691512799232,2020-03-22 00:07:00,Twitter Web App,0,129,None
2,Schnell Mal im Schatten von #COVID19 die #Über...,fradie,115,1241516670683840513,2020-03-22 00:06:55,Twitter for Android,0,0,None
3,RT @bpol_air_fra: Wir würden es natürlich etwa...,Alejandro Rodríguez,140,1241516626106859525,2020-03-22 00:06:44,Twitter for Android,0,184,None
4,RT @AfDimBundestag: .@Beatrix_vStorch erklärt ...,Für mein Land 🇩🇪 nur noch AfD 💙,140,1241516557953634306,2020-03-22 00:06:28,Twitter Web App,0,244,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
text      1000 non-null object
user      1000 non-null object
len       1000 non-null object
ID        1000 non-null object
Date      1000 non-null datetime64[ns]
Source    1000 non-null object
Like      1000 non-null object
RTs       1000 non-null object
dtypes: datetime64[ns](1), object(7)
memory usage: 62.6+ KB


In [5]:
df.describe()

,text,user,len,ID,Date,Source,Like,RTs
count,1000,1000,1000.0,1.000000e+03,1000,1000,1000.0,1000.0
unique,742,831,75.0,1.000000e+03,788,80,11.0,96.0
top,RT @Hoover_UK: We’re giving one lucky person t...,Nicole Clinton,140.0,1.068060e+18,2018-11-29 08:38:05,Twitter for iPhone,0.0,0.0
freq,33,8,566.0,1.000000e+00,7,244,930.0,326.0
first,NaN,NaN,NaN,NaN,2018-11-29 08:24:46,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2018-11-29 08:59:32,NaN,NaN,NaN


In [7]:
df.to_csv('/Users/andang/Documents/workspace/wirvsvirus/data/twitter_' + ((str) (datetime.date.today())).replace("-","")+ ((str) (datetime.datetime.now().time())).replace(":","").replace(".","") + ".csv", sep="\t")